In [21]:
from langchain_community.document_loaders import PyPDFLoader
import os
import getpass
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')



In [29]:
loader = PyPDFDirectoryLoader("/Users/kaitokikuchi/Library/CloudStorage/Dropbox/LLM-agriculture/PDFs")
docs = loader.load_and_split()

Illegal character in Name Object (b'/QEOWUY+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc-WinCharSetFFFF-H')
Illegal character in Name Object (b'/QEOWUY+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc')
Illegal character in Name Object (b'/CRDEAK+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc-WinCharSetFFFF-H')
Illegal character in Name Object (b'/CRDEAK+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc')


In [66]:


def make_embedder(model_type, save_path):
    if model_type == "sup-simcse-ja-base":
        model_name = "cl-nagoya/sup-simcse-ja-base"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
    else:  # Assuming OpenAIEmbeddings
        embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    
    fs = LocalFileStore(save_path)
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
    return cached_embedder

def save_embeddings_and_ids_with_faiss(all_documents, cached_embedder, save_path):
    db = FAISS.from_documents(all_documents, cached_embedder)
    db.save_local(save_path)



In [ ]:
model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(docs, cached_embedder, save_path)

Load from cached embedding

In [73]:
model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

model_name = "cl-nagoya/sup-simcse-ja-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)

In [75]:

def search(query: str):
    # Your search logic here
    print(query)
    embedded_query = embedding_model.embed_query(query)
    top_10_similar_docs = db.similarity_search_by_vector(embedded_query, k=10)
    for doc in top_10_similar_docs:
        print(doc)

In [77]:
search("令和4年度の福島県でのハスモンヨトウ発生状況は？")

令和4年度の福島県でのハスモンヨトウ発生状況は？
page_content='令和５年度 病害虫防除情報  \n \n令 和 ５ 年 ８ 月２９日 \n発表：福島県病害虫防除所  \n \n \n \n \n１ 対象作物：野菜類・花き類  \n２ 病害虫：ハスモンヨトウ  \n３ 対象地域： 全域 \n４ 発 生 量：やや多い  \n \n【発生状況等 】 \n(ア) 県内全域 に設置している ハスモンヨトウ のフェロモントラップへの誘殺数が 、８月中下\n旬に平年を大きく上回りました（図１～５ ）。 \n(イ) 仙台管区気象台が８月２４ 日に発表した１か月予報では、 気温が平年より高く、本種の\n発生が助長され ると考えられます。  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n 誘殺数が多くなっています。  \n防除対策を徹底し、被害を防ぎましょう。' metadata={'source': '/Users/kaitokikuchi/Library/CloudStorage/Dropbox/LLM-agriculture/PDFs/令和５年度 病害虫防除情報.pdf', 'page': 0}
page_content='令和４年度 病害虫防除情報  \n \n令 和 ４ 年 ９ 月 ２８日 \n発表：福島県病害虫防除所  \n \nハスモンヨトウ の誘殺数が多くなっています。  \n防除対策を徹底、被害 を防ぎましょう。  \n \n１ 対象作物：野菜類・花き類  \n２ 病害虫：ハスモンヨトウ  \n３ 対象地域： 中通り・会津 \n４ 発 生 量：やや多い  \n \n【発生状況等 】 \n(ア) 中通り及び会津に 設置している ハスモンヨトウ のフェロモントラップへの誘殺数が 、９\n月上中旬に平年を大きく上回りました（図１～４）。  \n(イ) ９月中旬に 中通り南部の露地キュウリで幼虫の寄生を確認しています。  \n(ウ) 仙台管区気象台が９月２２日 に発表した１か月予報では、 気温が平年より高く、本種の\n発生が助長され ると考えられます。' metadata={'source': '/Us

In [79]:

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [80]:

qa.invoke("令和4年度の福島県でのハスモンヨトウ発生状況は？")

{'query': '令和4年度の福島県でのハスモンヨトウ発生状況は？',
 'result': ' 令和4年度の福島県でのハスモンヨトウ発生状況は、やや多いとされています。中通り及び会津に設置しているフェロモントラップへの誘殺数が、9月上中旬に平年を大きく上回り、中通り南部の露地キュウリで幼虫の寄生が確認されています。また、仙台管区気象台の1か月予報では気温が平年より高く、本種の発生が助長されると考えられています。'}

In [39]:
query = "炭疽病の発生時期はいつ？"
qa.invoke(query)

{'query': '炭疽病の発生時期はいつ？',
 'result': ' 高温時期に発生しやすい。気温28℃の比較的高温条件で多発する。定植後秋口にすぐ症状が見られるものは、潜在感染した親株から採苗～育苗期に感染したと考えられる。'}

In [59]:
query = "イチゴのうどんこ病の防ぎ方を具体的に教えてください"
qa.invoke(query)

{'query': 'イチゴのうどんこ病の防ぎ方を具体的に教えてください',
 'result': ' 発生初期から防除を実施することが重要です。また、まん延すると防除が困難になるので、早めに対策を行うことが大切です。'}

In [81]:
query = "紫外線はうどんこ病を防げる？"
qa.invoke(query)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 5822 tokens (5566 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# Using OpenAI Embeddings
Note: as of 02/04/2024 Tiktoken has not been updated to include the [new OpenAI embedders](https://openai.com/blog/new-embedding-models-and-api-updates) so `Warning: model not found. Using cl100k_base encoding.` will be raised ([PR link](https://github.com/openai/tiktoken/pull/247))

In [67]:
model_type = "OpenAIEmbeddings"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(docs, cached_embedder, save_path)

In [69]:
model_type = "OpenAIEmbeddings"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

embedding_model = embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [71]:
query = "令和3年の福島県でのハスモンヨトウ発生状況は？"
qa.invoke(query)

AssertionError: 